## 1) Cargo los datos

In [1]:
import pandas as pd 
from funpymodeling.exploratory import freq_tbl, status

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_csv("modulos/data/customer_dataset.csv", sep=',')

In [4]:
data

,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionAmount,transactionFailed,fraudulent,emailDomain,emailProvider,customerIPAddressSimplified,sameCity
0,18.0,pending,True,card,JCB 16 digit,Citizens First Banks,18,False,False,com,yahoo,only_letters,yes
1,26.0,fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,26,False,True,com,yahoo,only_letters,no
2,45.0,fulfilled,False,card,VISA 16 digit,Vertex Bancorp,45,False,False,com,yahoo,digits_and_letters,no
3,23.0,fulfilled,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,23,False,False,com,yahoo,digits_and_letters,no
4,43.0,fulfilled,False,card,Mastercard,Vertex Bancorp,43,True,True,com,other,only_letters,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,NaN,NaN,NaN,NaN,NaN,NaN,25,True,NaN,weird,weird,only_letters,unknown
619,NaN,NaN,NaN,NaN,NaN,NaN,25,True,NaN,weird,weird,only_letters,unknown
620,NaN,NaN,NaN,NaN,NaN,NaN,25,False,NaN,weird,weird,only_letters,unknown
621,NaN,NaN,NaN,NaN,NaN,NaN,19,False,NaN,weird,weird,only_letters,unknown


## 3) Discretizacion 

Antes se realizará una copia del dataframe data para evaluar si los puntos de corte se guardan y leen corerctamente

In [5]:
probando = data.copy()

Se discretiza 'orderAmount' por igual frecuencia y por igual rango.

In [6]:
data['orderAmount'], saved_bins_order = pd.qcut(data['orderAmount'], q=5, duplicates='drop', retbins=True)

Se guardan los puntos de corte.

In [7]:
import pickle

In [8]:
with open('modulos/data/saved_bins_order.pickle', 'wb') as handle: 
    pickle.dump(saved_bins_order, handle, protocol=pickle.HIGHEST_PROTOCOL)

Se discretiza 'transactionAmount' por igual frecuencia y por igual rango.

In [9]:
data['transactionAmount'], saved_bins_transaction = pd.qcut(data['transactionAmount'], q=4, duplicates = 'drop', retbins = True)

Se guardan los puntos de corte. 

In [10]:
with open('modulos/data/saved_bins_transaction.pickle', 'wb') as handle: 
    pickle.dump(saved_bins_transaction, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Evaluo que los puntos de corte se han guardado correctamente

In [11]:
with open('modulos/data/saved_bins_order.pickle', 'rb') as handle: 
    new_saved_bins_order = pickle.load(handle)

In [12]:
with open('modulos/data/saved_bins_transaction.pickle', 'rb') as handle: 
    new_saved_bins_transaction = pickle.load(handle)

In [13]:
probando["orderAmount"] = pd.cut(
    probando["orderAmount"],
    bins = new_saved_bins_order,
    include_lowest=True
)

In [14]:
probando["transactionAmount"] = pd.cut(
    probando["transactionAmount"],
    bins = new_saved_bins_transaction, 
    include_lowest = True
)

In [15]:
probando.head(3)

,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionAmount,transactionFailed,fraudulent,emailDomain,emailProvider,customerIPAddressSimplified,sameCity
0,"(9.999, 18.4]",pending,True,card,JCB 16 digit,Citizens First Banks,"(9.999, 21.0]",False,False,com,yahoo,only_letters,yes
1,"(18.4, 30.0]",fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,"(21.0, 34.0]",False,True,com,yahoo,only_letters,no
2,"(39.0, 47.0]",fulfilled,False,card,VISA 16 digit,Vertex Bancorp,"(34.0, 45.0]",False,False,com,yahoo,digits_and_letters,no


## Preparacion de datos

#### Cuento los valores unicos que aparecen en paymentMethodIssuer, paymentMethodProvider, paymentMethodType y fraudulent.

In [16]:
data['paymentMethodIssuer'].value_counts()

paymentMethodIssuer
Her Majesty Trust           43
Vertex Bancorp              37
Fountain Financial Inc.     35
His Majesty Bank Corp.      33
Bastion Banks               29
Bulwark Trust Corp.         29
Citizens First Banks        28
Grand Credit Corporation    27
Solace Banks                27
Rose Bancshares             25
B                            7
e                            5
c                            4
r                            3
                             2
n                            2
x                            2
o                            2
a                            1
p                            1
Name: count, dtype: int64

Se puede observar que hay valores extraños, por lo que los agrupamos manualmente.

In [17]:
weird_payment_method = ["B", "e", "c", "r", " ", "n", "x", "o", "a", "p"]

In [18]:
for payment_method in weird_payment_method:
    data['paymentMethodIssuer'] = data['paymentMethodIssuer'].replace(payment_method, 'weird')

In [19]:
data['paymentMethodIssuer'].value_counts()

paymentMethodIssuer
Her Majesty Trust           43
Vertex Bancorp              37
Fountain Financial Inc.     35
His Majesty Bank Corp.      33
Bastion Banks               29
weird                       29
Bulwark Trust Corp.         29
Citizens First Banks        28
Solace Banks                27
Grand Credit Corporation    27
Rose Bancshares             25
Name: count, dtype: int64

In [20]:
data['paymentMethodProvider'].value_counts()

paymentMethodProvider
JCB 16 digit                   65
VISA 16 digit                  57
Voyager                        36
Diners Club / Carte Blanche    34
Maestro                        32
VISA 13 digit                  32
Discover                       25
American Express               22
JCB 15 digit                   20
Mastercard                     19
Name: count, dtype: int64

In [21]:
data['paymentMethodType'].value_counts()

paymentMethodType
card         242
apple pay     36
paypal        36
bitcoin       28
Name: count, dtype: int64

In [22]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,orderAmount,145,0.232745,0,0.000000,5,category
1,orderState,145,0.232745,0,0.000000,3,object
2,paymentMethodRegistrationFailure,281,0.451043,302,0.484751,2,object
3,paymentMethodType,281,0.451043,0,0.000000,4,object
4,paymentMethodProvider,281,0.451043,0,0.000000,10,object
5,paymentMethodIssuer,281,0.451043,0,0.000000,11,object
6,transactionAmount,0,0.000000,0,0.000000,4,category
7,transactionFailed,0,0.000000,455,0.730337,2,bool
8,fraudulent,455,0.730337,107,0.171750,2,object
9,emailDomain,0,0.000000,0,0.000000,6,object


In [23]:
data['fraudulent'].value_counts()

fraudulent
False    107
True      61
Name: count, dtype: int64

In [24]:
data['fraudulent'] = data['fraudulent'].fillna(value= 'warning')

In [25]:
data['fraudulent'].value_counts()

fraudulent
warning    455
False      107
True        61
Name: count, dtype: int64

Me aseguro que no exista ningun conflicto por falta de compatibilidad

In [26]:
data['fraudulent'] = data['fraudulent'].astype(str)

In [27]:
class_map = {'False': 0, 'True': 1, 'warning': 2}
data['fraudulent'] = data['fraudulent'].map(class_map)

In [28]:
data.head()

,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionAmount,transactionFailed,fraudulent,emailDomain,emailProvider,customerIPAddressSimplified,sameCity
0,"(9.999, 18.4]",pending,True,card,JCB 16 digit,Citizens First Banks,"(9.999, 21.0]",False,0,com,yahoo,only_letters,yes
1,"(18.4, 30.0]",fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,"(21.0, 34.0]",False,1,com,yahoo,only_letters,no
2,"(39.0, 47.0]",fulfilled,False,card,VISA 16 digit,Vertex Bancorp,"(34.0, 45.0]",False,0,com,yahoo,digits_and_letters,no
3,"(18.4, 30.0]",fulfilled,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,"(21.0, 34.0]",False,0,com,yahoo,digits_and_letters,no
4,"(39.0, 47.0]",fulfilled,False,card,Mastercard,Vertex Bancorp,"(34.0, 45.0]",True,1,com,other,only_letters,no


#### Tratamiento de datos faltantes

In [29]:
data['orderAmount'].value_counts()

orderAmount
(30.0, 39.0]     98
(39.0, 47.0]     97
(9.999, 18.4]    96
(18.4, 30.0]     96
(47.0, 353.0]    91
Name: count, dtype: int64

In [30]:
data['orderAmount'] = data['orderAmount'].cat.add_categories("desconocido")
data['orderAmount'] = data['orderAmount'].fillna(value="desconocido")

In [31]:
data['orderAmount'].value_counts()

orderAmount
desconocido      145
(30.0, 39.0]      98
(39.0, 47.0]      97
(9.999, 18.4]     96
(18.4, 30.0]      96
(47.0, 353.0]     91
Name: count, dtype: int64

## One hot encoding

In [32]:
data_ohe = pd.get_dummies(data)

In [33]:
status(data_ohe)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,transactionFailed,0,0.0,455,0.730337,2,bool
1,fraudulent,0,0.0,107,0.171750,3,int64
2,"orderAmount_(9.999, 18.4]",0,0.0,527,0.845907,2,bool
3,"orderAmount_(18.4, 30.0]",0,0.0,527,0.845907,2,bool
4,"orderAmount_(30.0, 39.0]",0,0.0,525,0.842697,2,bool
5,"orderAmount_(39.0, 47.0]",0,0.0,526,0.844302,2,bool
6,"orderAmount_(47.0, 353.0]",0,0.0,532,0.853933,2,bool
7,orderAmount_desconocido,0,0.0,478,0.767255,2,bool
8,orderState_failed,0,0.0,573,0.919743,2,bool
9,orderState_fulfilled,0,0.0,223,0.357945,2,bool


In [34]:
data_ohe_without_fraudulent = data_ohe.drop(['fraudulent'], axis=1)

In [35]:
with open('modulos/data/categories_ohe_without_fraudulent.pickle', 'wb') as handle: 
    pickle.dump(data_ohe_without_fraudulent.columns, handle, protocol = pickle.HIGHEST_PROTOCOL)

## Guardo dataset

In [36]:
filename = "modulos/data/ohe_customer_dataset.csv"
data_ohe.to_csv(filename, index= False)